In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import sys
import gym
from importlib import reload
from configparser import ConfigParser
from datetime import datetime
import tensorflow as tf
import time


# get the relative path
pref                = os.getcwd()

os.environ["RL_PATH"]   = pref

#os.environ["RL_PATH"] = "/Users/ankitgupta/Documents/git/anks/Books/ReinforcementLearning/DeepQLearning"
#pref = os.environ["RL_PATH"]

if f'{pref}/RLLibrary' not in sys.path:
    sys.path.append(f'{pref}/RLLibrary')


In [2]:


# importing custom libraries
from ActionSelection import ActionExploration
from ConfigReader import Config
#from RLAgents import QLearningAgent, FittedQAgent, DQN, DoubleDQN
import PolicyGradientAgents
import utils as RLUtils
from Agents.PolicyGradient import Reinforce, ActorCritic
from Agents.PolicyGradient import discountRewards
import loggingConfig as loggingConfig


logger = loggingConfig.logging
logger.getLogger("PG_Train")


<Logger PG_Train (INFO)>

In [3]:
reload(Reinforce)

<module 'Agents.PolicyGradient.Reinforce' from '/Users/ankitgupta/Documents/git/anks/MachineLearning/ReinforcementLearning/RLLibrary/Agents/PolicyGradient/Reinforce.py'>

In [4]:
configFile  = os.path.join(pref, "Configs.ini" )
savePath    = os.path.join(os.environ["RL_PATH"], "models" )
_time       = datetime.now().strftime("%Y%m%d%H%M")

env = gym.make("CartPole-v1")
state = env.reset()


In [6]:
modelParams = {"Name": "Policy", "NetworkShape": [24, 12], "learning_rate": 0.01, \
                "optimizer": "ADAM", "loss": "categorical_crossentropy", }
valueParams = {"Name_value": "Value", "NetworkShape_value": [24, 12], "learning_rate_value": 0.0001, \
                "optimizer_value": "ADAM", "loss_value": "mse", }






In [43]:
reload(ActorCritic)

<module 'Agents.PolicyGradient.ActorCritic' from '/Users/ankitgupta/Documents/git/anks/MachineLearning/ReinforcementLearning/RLLibrary/Agents/PolicyGradient/ActorCritic.py'>

In [44]:
ReinforceAgent   = ActorCritic.ActorCritic(env = env, configFile = configFile, **modelParams)


In [45]:
ReinforceAgent.SharedNetwork.optimizer

In [46]:
ReinforceAgent.SharedNetwork.summary()

Model: "SharedDesign"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
dense_8 (Dense)                 (None, 24)           120         input_3[0][0]                    
__________________________________________________________________________________________________
dense_9 (Dense)                 (None, 12)           300         dense_8[0][0]                    
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 2)            26          dense_9[0][0]                    
_______________________________________________________________________________________

In [47]:
# lets run for 1 episode

In [48]:
Agent = ReinforceAgent
loss = []
mode = "TRAIN"
for _thisepisode in tqdm(range(Agent.NbEpisodesTrain)):

    # reset the environment
    _currentState = env.reset()
    

    _episodicReward = 0
    _dead = False
    _thisstepsTaken = 0


    while _thisstepsTaken <= Agent.MaxSteps:

        _starttime = time.perf_counter()

        # get the action from agent
        if "REINFORCE" in Agent.Name :
            # only actor based methods. Network doesnt return critic values
            action, actionProb = Agent.getAction(_currentState, mode = mode)
        else:
            action, actionProb, value = Agent.getAction(_currentState, mode = mode)            

        # perform the action
        _nextState, _reward, _dead, _info = env.step(action)

        # record into memory
        if "REINFORCE" in Agent.Name :

            Agent.updateMemory(_currentState, action, _reward, _nextState, _dead, actionProb)
        else:
            Agent.updateMemory(_currentState, action, _reward, _nextState, _dead, actionProb, value)

                
        # update States
        _currentState = _nextState
        _thisstepsTaken += 1

        _episodicReward += _reward

        # if game over, then exit the loop
        if _dead == True:

            #if (_thisepisode+1)%update_frequency == 0:
            Agent.train()

            # ---- For logging ------
            # In case of Neural networks, create tensorboard flow
            Agent.updateLoggerInfo(episodeCount = _thisepisode, episodicReward = _episodicReward, \
                                    episodicStepsTaken = _thisstepsTaken, mode = "TRAIN")

            _endtime = time.perf_counter()

            logger.info(f'Episode: {_thisepisode+1} Steps: {_thisstepsTaken} Reward:{_episodicReward} Time taken: {round(_endtime - _starttime,2)} secs ')

            break



  0%|          | 0/2000 [00:00<?, ?it/s]


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



  0%|          | 0/2000 [00:00<?, ?it/s]


NameError: name 'numpy' is not defined

In [16]:
_currentState

array([ 0.0050496 , -0.02645206,  0.01865667, -0.008793  ])

In [18]:
_currentState = _currentState.reshape([1, _currentState.shape[0]])
_currentState

array([[ 0.0050496 , -0.02645206,  0.01865667, -0.008793  ]])

In [27]:
act, val = Agent.SharedNetwork(_currentState)

In [34]:
act /= tf.reduce_sum(act)

In [42]:
val.numpy()

array([[-0.00070665]], dtype=float32)

In [30]:
tf.reduce_sum(act)


<tf.Tensor: id=1131, shape=(), dtype=float32, numpy=1.0>

In [40]:
action             = np.random.choice(Agent.env.action_space.n, p = act.numpy()[0])
action

0

In [39]:
# for all experience in batchsize
curStates       = np.vstack(list(list(zip(*Agent.memory)))[0])
actions         = np.vstack(list(list(zip(*Agent.memory)))[1])
nextStates      = np.vstack(list(list(zip(*Agent.memory)))[3])
rewards         = np.vstack(list(list(zip(*Agent.memory)))[2])
done            = np.vstack(list(list(zip(*Agent.memory)))[4])

actionProb      = np.vstack(list(list(zip(*Agent.memory)))[5])


IndexError: list index out of range

In [28]:
# compute the discounted rewards for the entire episode and normalize it
discounted_rewards = discountRewards(rewards,  discountfactor=Agent.discountfactor)


In [30]:
discounted_rewards = (discounted_rewards - np.mean(discounted_rewards))/ (np.std(discounted_rewards) + 1e-7)            # to avoid division by 0

In [33]:
discounted_rewards[4][0]

0.9905667496168143

In [61]:
losses = []

with tf.GradientTape() as tape:
    for index, sample in enumerate(Agent.memory):

        # get log prob
        state = tf.Variable([sample[0]], trainable=True, dtype=tf.float64)
        prob = Agent.PolicyNetwork.model(state, training = True)
        action = sample[1]
        actionProb = prob[0, action]
        logProb = tf.math.log(actionProb)

        #actionProb = sample[5]
        #logProb = np.log(actionProb)
        sampleloss = logProb * discounted_rewards[index][0]

        losses.append(-sampleloss)      # this is negative, since we are interested in gradient ascent

    networkLoss = sum(losses)
    
    grads = tape.gradient(networkLoss, Agent.PolicyNetwork.model.trainable_variables)
    Agent.PolicyNetwork.optimizer.apply_gradients(zip(grads, Agent.PolicyNetwork.model.trainable_variables))



In [51]:
sample

(array([-0.12274747, -1.90964119,  0.19205285,  2.89212446]),
 1,
 1.0,
 array([-0.16094029, -1.7161645 ,  0.24989534,  2.66336226]),
 True,
 array([0.7354947 , 0.26450533], dtype=float32))

In [50]:
prob

<tf.Tensor: id=5343, shape=(1, 2), dtype=float32, numpy=array([[0.7354947, 0.2645053]], dtype=float32)>

In [52]:
action = sample[1]
actionProb = prob[0, action]
actionProb

<tf.Tensor: id=5352, shape=(), dtype=float32, numpy=0.2645053>

In [48]:
logProb

<tf.Tensor: id=5344, shape=(1, 2), dtype=float32, numpy=array([[-0.307212, -1.329894]], dtype=float32)>

AttributeError: 'numpy.dtype' object has no attribute 'is_floating'

In [ ]:

# compute the discounted rewards for the entire episode and normalize it
discounted_rewards = discountRewards(rewards,  discountfactor=self.discountfactor)
if self.normalizeRewards:
    discounted_rewards = (discounted_rewards - np.mean(discounted_rewards))/ (np.std(discounted_rewards) + 1e-7)            # to avoid division by 0


#TODO -----> add eager execution
losses = []

with tf.GradientTape() as tape:
    for curStates, actions, nextStates, rewards, done, actionProb in self.memory:

        # get log prob
        logProb = np.log(actionProb)
        sampleloss = logProb * discounted_rewards

        losses.append(-sampleloss)      # this is negative, since we are interested in gradient ascent

    networkLoss = np.sum(losses)

grads = tape.gradient(networkLoss, self.PolicyNetwork.model.trainable_variables)
self.PolicyNetwork.optimizer.apply_gradients(zip(grads, model.trainable_variables))


In [48]:

# for all experience in batchsize
curStates       = np.vstack(list(list(zip(*Agent.memory)))[0])
actions         = np.vstack(list(list(zip(*Agent.memory)))[1])
nextStates      = np.vstack(list(list(zip(*Agent.memory)))[3])
rewards         = np.vstack(list(list(zip(*Agent.memory)))[2])
done            = np.vstack(list(list(zip(*Agent.memory)))[4])

actionProb      = np.vstack(list(list(zip(*Agent.memory)))[5])


# Get X
try:
    Agent.env.observation_space.n
    X_train = getOneHotrepresentation(curStates, num_classes=Agent.inputShape)
except:
    X_train = curStates    


In [49]:
X_train[:4], actionProb[:4]

(array([[-0.03113103, -0.02077829,  0.00398832,  0.00614775],
        [-0.03154659,  0.17428624,  0.00411127, -0.28527415],
        [-0.02806087, -0.0208941 , -0.00159421,  0.0087026 ],
        [-0.02847875,  0.17425067, -0.00142016, -0.28448289]]),
 array([[0.5012765 , 0.49872348],
        [0.5122483 , 0.48775175],
        [0.50089973, 0.49910033],
        [0.51175475, 0.4882453 ]], dtype=float32))

In [50]:
discounted_rewards = discountRewards(rewards,  discountfactor=Agent.discountfactor)
discounted_rewards[:4]

[array([14.85422289]),
 array([13.99416454]),
 array([13.12541872]),
 array([12.2478977])]

In [51]:
if Agent.baseline.upper() == "VALUE":
    value = Agent.ValueNetwork.model.predict(X_train)
    G = discounted_rewards - value


elif Agent.baseline.upper() == "NORMALIZE":
    G = discounted_rewards
    G = (G - np.mean(G))/ (np.std(G) + 1e-7)            # to avoid division by 0

G[:4]    

array([[1.58898873],
       [1.38696682],
       [1.1829043 ],
       [0.97678053]])

In [52]:
gradient = np.subtract(RLUtils.getOneHotrepresentation(actions,Agent.env.action_space.n ), actionProb)
gradient[:4]

array([[-0.5012765 ,  0.5012765 ],
       [ 0.48775172, -0.48775175],
       [-0.50089973,  0.5008997 ],
       [ 0.48824525, -0.4882453 ]], dtype=float32)

In [53]:
gradient *= G 
gradient[:4]

array([[-0.7965227 ,  0.7965227 ],
       [ 0.67649543, -0.6764955 ],
       [-0.5925164 ,  0.59251636],
       [ 0.47690845, -0.4769085 ]], dtype=float32)

In [54]:
gradient *= Agent.policy_learning_rate
gradient[:4]

array([[-7.9652265e-05,  7.9652265e-05],
       [ 6.7649540e-05, -6.7649547e-05],
       [-5.9251641e-05,  5.9251634e-05],
       [ 4.7690843e-05, -4.7690850e-05]], dtype=float32)

In [40]:
import copy

In [55]:
# old model
old = tf.keras.models.clone_model(Agent.PolicyNetwork.model)

In [56]:

# updating actual probabilities (y_train) to take into account the change in policy gradient change
# \theta = \theta + alpha*rewards * gradient
y_train = actionProb + np.vstack(gradient)



# update policy and also learn the value function

# 1. Update policy
history = Agent.PolicyNetwork.model.train_on_batch(X_train, y_train)

Agent.memory = []

In [58]:
old.predict(X_train[:4])

array([[0.49950963, 0.50049037],
       [0.5345103 , 0.46548966],
       [0.49908236, 0.5009177 ],
       [0.53361577, 0.4663842 ]], dtype=float32)

In [59]:
Agent.PolicyNetwork.model.predict(X_train[:4])

array([[0.5013335 , 0.49866652],
       [0.5123426 , 0.4876574 ],
       [0.5009569 , 0.4990431 ],
       [0.51184905, 0.48815095]], dtype=float32)

In [1]:
import tensorflow_probability as tfp

ModuleNotFoundError: No module named 'tensorflow_probability'